In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dataframe_image as dfi
from datetime import datetime
import networkx as nx
import scipy
import itertools

In [2]:
file = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\1997-01-01-2023-07-18-Eastern_Africa-Somalia.csv"
df = pd.read_csv(file)
len(df)

41397

In [3]:
df=df[df['event_type'] != 'Protests']
df=df[df['event_type'] != 'Strategic developments']
len(df)

36995

In [4]:
v= df["event_date"]
v=v.values
dt = [datetime.strptime(v[i], "%d %B %Y") for i in range(len(v))]
q=[]

for i in range(len(dt)):
    q.append(datetime.timestamp(dt[i]))
    
df.insert(loc=3, column='date_timestamp', value=q)
df = df.sort_values("date_timestamp")

df['event_date'] = pd.to_datetime(df['event_date'])
df=df.set_index('event_date') 

In [5]:
conflict=df.groupby([pd.Grouper(freq='40320min'),"admin1"]).count()
conflict=conflict.reset_index(level=[0, 1])
conflict=conflict[['event_date','admin1','year']]
conflict=conflict.rename(columns={'year': 'conflicts','event_date': 'time'})

conflict['datetime_column'] = pd.to_datetime(conflict['time'])

# Aggregate the datetime objects by month
aggregated_data = conflict.groupby([pd.Grouper(key='time', freq='M'),'admin1'])['conflicts'].sum()

conf=aggregated_data.to_frame()

In [6]:
dates = conf.index.get_level_values('time').unique()
districts = conf.index.get_level_values('admin1').unique()
all_combinations = pd.MultiIndex.from_product([dates, districts], names=['time', 'admin1'])

# Reindex the DataFrame with all combinations
df1 = conf.reindex(all_combinations, fill_value=0)

# Reset the index
df1 = df1.reset_index()

In [7]:
conflicts = df1.sort_values(by=['time', 'admin1'], ascending=[True, True])
conflicts.reset_index(drop=True, inplace=True)

In [20]:
file = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\temp_pre_new.csv"
temp_pre = pd.read_csv(file)
#rename columns
temp_pre=temp_pre.rename(columns={'T2M_MAX':'tmx','PRECTOTCORR': 'pre','region': 'admin1'})

In [21]:
temp_pre.head()

,time,admin1,pre,tmx
0,199701.0,Awdal,0.312500,30.277500
1,199701.0,Bakool,0.006250,36.488747
2,199701.0,Bari,0.072500,29.962084
3,199701.0,Bay,0.001429,38.415000
4,199701.0,Galgaduud,0.023529,34.728237


In [22]:
district1 = 'Lower Shabelle'  # Replace with the name of the first neighboring district
district2 = 'Middle Shabelle'  # Replace with the name of the second neighboring district

# Step 2: Calculate the mean temp for the neighboring districts
mean_t = temp_pre[(temp_pre['admin1']==district1) | (temp_pre['admin1']==district2)].groupby('time')['tmx'].mean()

# Step 2: Calculate the mean prec for the neighboring districts
mean_p = temp_pre[(temp_pre['admin1']==district1) | (temp_pre['admin1']==district2)].groupby('time')['pre'].mean()

# Step 3: Create a new DataFrame with the additional district and the mean conflict values
additional_district = 'Banadir'  # Replace with the name of the additional district
new_data = pd.DataFrame({ 'admin1': additional_district, 'tmx': mean_t, 'pre': mean_p})
new_data=new_data.reset_index()

# Step 4: Append the new DataFrame to the original DataFrame
df3 = pd.concat([temp_pre, new_data])

In [23]:
temp_pre=df3.sort_values(by=['time', 'admin1'], ascending=[True, True]).reset_index(drop=True)

#make all enlements in array integers
temp_pre['time']=temp_pre['time'].astype('int64').astype(int)

#add a '-' between year and month in temp_pre['time']
temp_pre['month_year']=temp_pre['time'].astype(str).str[:-2] + '-' + temp_pre['time'].astype(str).str[-2:]

#from the format '1997-01' get '01'
temp_pre['month'] = temp_pre['month_year'].str[5:7]
#remove the rows that have the month 13 in column 'month'
temp_pre=temp_pre[temp_pre['month'] != '13'].reset_index(drop=True)

In [24]:
conflicts.reset_index(drop=True, inplace=True)

In [25]:
conflicts['month_year'] = conflicts['time'].dt.strftime('%Y-%m').values

In [26]:
# Perform an outer merge on the month_year column, which includes all month and year combinations from both dataframes
merged_df = pd.merge(temp_pre, conflicts, on=['month_year','admin1'], how='outer')

# Fill missing values with zeros
merged_df = merged_df.fillna(0)

# Remove the temporary column
merged_df.drop(['time_x','time_y'], axis=1, inplace=True)

In [27]:
#reorder columns
merged_df = merged_df[['month_year','admin1','tmx','pre','conflicts']]
#rename columns
merged_df=merged_df.rename(columns={'month_year':'time'})

In [29]:
merged_df=merged_df[merged_df['tmx'] != 0].reset_index(drop=True)

In [30]:
merged_df.to_csv('temp-pre-conf-new.csv', index=False)